# Pandas e Plotly

Exemplos de análise de dados dos CSV gerados pelo programa `Heurística_HDB`.

Utilizaremos os recursos avançados de análise de dataframes com pandas e em seguida vamos utilizar a biblioteca Plotly para gerar gráficos interativos.

---


## Importando as bibliotecas e módulos

In [ ]:
# import pandas and plotly modules
import pandas as pd
import plotly.express as px
# import module for dealing with your OS
import os

## Inserindo caminho para o CSV e termo para salvar o arquivo

In [ ]:
# input CSV path
csv_path = input("Enter CSV path: ")
# input name of output file
output_file = input("Enter output file name: ")
# upper case the output file name
output_file = output_file.upper()

## Criando o dataframe

A partir do caminho do CSV, vamos utilizar o método `pd.read_csv()` para gerar o dataframe.

In [ ]:
# Create dataframe using pandas `pd.read_csv(<path to csv>)`
df = pd.read_csv(f'{csv_path}')
# we can use `.head()` to get the first 5 rows
print(df.head())

## Analisando cada coluna

É importante conhecer o dataframe. Pra isso vamos olhar com atenção os títulos e tipos de cada coluna utilizando `.columns` e `.dtypes`

In [ ]:
# we can use the `.columns` property to get the column names
# and analyze the dataframe
print(df.columns)
# we can use `dtypes` property to get the data types of each column
print(df.dtypes)

## Remover uma coluna

Para remover uma coluna do dataframe usamos `.drop()`. 

In [ ]:
# use drop to remove the `Unnamed: 0` column
# axis = 0 means rows, axis = 1 means columns
# inplace = True means it will make the changes in place
df.drop('Unnamed: 0', axis=1, inplace=True)
print(df.columns)

## Alterar valores de uma coluna

É possível alterar strings numa determinada coluna utilizando _regular expressions_.

Vamos remover as datas que seguem todos os _acervos_ no padrão `- 1999 a 2000` e `- 1999`.

In [ ]:
# use regex to find the pattern of date and remove it
df['Acervo'] = df['Acervo'].str.replace('- \d{4} a \d{4}', '', regex=True)
df['Acervo'] = df['Acervo'].str.replace('- \d{4}', '', regex=True)

# print head
print(df.head())

## Filtrar e analisar por colunas específicas

É possível agrupar os valores de uma determinada coluna através do `groupby()`.

O Pandas permite agrupar por mais de uma coluna ao mesmo tempo.

Nos exemplos, vamos filtrar pelas colunas 'Ano' e 'Acervo'.

In [ ]:
# group by column 'Acervo'
df_acervos = df.groupby('Acervo')
# use `.size()` to get the number of rows in each group
print(df_acervos.size())

>OBS: Importante reparar que nesse caso criamos um novo dataframe com os dados agrupados pela coluna 'Acervo'.

In [ ]:
# use sort_values to sort the dataframe
df_acervos = df_acervos.size().sort_values(ascending=False)
print(df_acervos.head())

### Colocar o index como uma nova coluna e inserir um index novo

In [ ]:
# use reset_index to reset the index
df_acervos = df_acervos.reset_index()
print(df_acervos.head())

### Alterar o nome da coluna de contagem total

In [ ]:
# use `.rename()` to rename the column and inplace = True
df_acervos.rename(columns={0: 'Total de Ocorrências'}, inplace=True)
print(df_acervos.head())

#### df_acervos

Esse dataframe contém o a quatidade total de ocorrências por jornal em ordem decrescente.

Ficaria bom num gráfico de barras, né? Veremos mais abaixo.

### Filtrar usando duas colunas

In [ ]:
# filter by 'Ano' and 'Acervo'
# groupby is a function that groups data by a column and returns a dataframe
# size is the number of rows
df_ano_acervo = df.filter(items=['Ano', 'Acervo'])\
    .groupby(['Ano', 'Acervo'])\
    .size()
print(df_ano_acervo.head())

In [ ]:
# use reset_index() to reset the index 
df_ano_acervo = df_ano_acervo.reset_index()
# sort the dataframe by 'Ano'
df_ano_acervo = df_ano_acervo.sort_values(by=['Ano'])
# rename column '0' to 'quant_oco_ano'
df_ano_acervo.rename(columns={0: 'quant_oco_ano'}, inplace=True)
print(df_ano_acervo.head())
print(df_ano_acervo.columns)

#### df_ano_acervo

Esse dataframe contém a quatidade total de ocorrências por jornal em cada ano, ordenados por ano (crescente).

---

## Plotly

Vamos utilizar os dataframes criar gráficos interativos com [plotly](https://plotly.com/)

### df_acervos

Gráfico de barras com `df_acervos`.

In [ ]:
# create a bar chart with the df_acervos dataframe
fig = px.bar(df_acervos, x='Acervo', y='Total de Ocorrências', color= 'Acervo', height=800, width=1000,
                 hover_data=[df_acervos['Acervo'], df_acervos['Total de Ocorrências']],
                 title=f'{output_file} - Jornal X Total de ocorrências',
                 )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    legend=dict(orientation="v"),
    showlegend=False,
    xaxis=dict(title="Acervos"),
    yaxis=dict(title="Total de Ocorrências"),
    title_font_size=16,
    title_pad=dict(l=50, r=0, t=0, b=0),
)
# update xaxes
fig.update_xaxes(
        tickangle = 60,
        title_standoff = 25,
        nticks=20, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.update_yaxes(
        tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )

fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_acervos_oco_bar.html')
#save the figure as a html file
fig.write_html(fig_path)


In [ ]:
# scatter plot with plotly express using the count of the number of times the 'Acervo' was repeated
fig = px.scatter(df_ano_acervo, x= 'Acervo' , y='Ano', color='Ano', height=1000, width=1200,
                 hover_data=[df_ano_acervo['Ano'], df_ano_acervo['Acervo'], df_ano_acervo['quant_oco_ano']],
                 title=f"{output_file} - Ano X Acervo",
                 size='quant_oco_ano',
                 )
# config the layout
fig.update_layout(
    hovermode="closest",
    legend=dict(orientation="h", xanchor="center", yanchor="top"),
    showlegend=True,
    xaxis=dict(title="Jornais"),
    yaxis=dict(title="Anos em que há ao menos uma ocorrência"),
    title_font_size=20,
    title_pad=dict(l=300, r=0, t=0, b=0)
)
# update xaxes
fig.update_xaxes(
        tickangle = 60,
        title_standoff = 25,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.update_yaxes(
        nticks=20, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )

fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_acervo.html')
#save the figure as a html file
fig.write_html(fig_path)

In [ ]:
# scatter plot with plotly express using the count of the number of times the 'Acervo' was repeated
fig = px.scatter(df_ano_acervo, x= 'Ano' , y='Acervo', color='Acervo', height=1000, width=1200,
                 hover_data=[df_ano_acervo['Ano'], df_ano_acervo['Acervo'], df_ano_acervo['quant_oco_ano']],
                 title=f"{output_file} - Ano X Acervo",
                 size='quant_oco_ano',
                 )
# config the layout
fig.update_layout(
    autosize=True,
    hovermode="closest",
    legend=dict(orientation="v"),
    showlegend=False,
    xaxis=dict(title="Anos em que há ao menos uma ocorrência"),
    yaxis=dict(title="Jornais"),
    title_font_size=20,
    title_pad=dict(l=600, r=0, t=0, b=0),
)
# update xaxes
fig.update_xaxes(
        tickangle = 60,
        title_standoff = 25,
        nticks=20, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.update_yaxes(
        tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )

fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_anos.html')
#save the figure as a html file
fig.write_html(fig_path)

### Quantidade ocorrências de cada jornal por ano

In [ ]:
# dataframe with the number of times the 'Acervo' was repeated in each year
df3  = df.filter(items=['Ano', 'Acervo'])\
    .groupby(['Acervo', 'Ano'])\
    .size()
# use reset_index() to change the index to the new column
df3 = df3.reset_index()
# rename column '0' to 'quant_oco_ano'
df3.rename(columns={0: 'quant_oco_ano'}, inplace=True)
print(df3.head())

In [ ]:
# filter by 'Ano' between 1901 and 1940 usinf .query()
df4 = df3.query('Ano >=1901 and Ano <=1940')
print(df4.head())


### Gráfico de linha

Vamos utilizar o `df4` ( coluna 'Ano' filtrada entre 1901 e 1940) para testar um gráfico de linha com plotly

In [ ]:
#line plot with plotly express using the count of the number of times the 'Acervo' was repeated between 1901 and 1940
fig = px.line(df4, x='Ano', y='quant_oco_ano', color='Acervo', height=1000, width=1200,
                 hover_data=[df4['Ano'], df4['Acervo'], df4['quant_oco_ano']],
                 title=f'{output_file} - Ano X Acervo',
                 markers=True,
                 )
fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_acervo_filtered_line.html')
#save the figure as a html file
fig.write_html(fig_path)

### Analisando ocorrências por década

Usar `groupby` e `sum()` para chegar ao total de ocorrências por ano

In [ ]:
#sum values of 'quant_oco_ano' in each decade and use reset_index() to change the index to the new column
df5 = df3.groupby(['Ano'])['quant_oco_ano'].sum().reset_index()
print(df5.head())

In [ ]:
# get 'quant_oco_ano' column from df5 and put it in a list using .tolist()
lista_valores = df5['quant_oco_ano'].tolist()
print(f'Total de anos com ocorrências: {len(lista_valores)}')
# get the total sum of ints in lista_valores
soma = sum(lista_valores)
print(f'Total de ocorrências: {soma}')

In [ ]:
# function to get the sum of 'quant_oco_ano' in each decade
def get_sum_decade(df, ano_inicial, ano_final):
    return df.query(f'Ano >= {ano_inicial} and Ano <= {ano_final}')['quant_oco_ano'].sum()

In [ ]:
# Creates a dictionary with beggining and end of each decade
# decades_dic = {'decade_beggining': 'decade_end'}
decades_dic = {1850: 1859, 1860:1869, 1870:1879, 1880:1889, 1890:1899, 1900:1909, 1910:1919, 1920:1929, 1930:1939, 1940:1949, 1950:1959, 1960:1969, 1970:1979, 1980:1989, 1990:1999, 2000:2009}
# iterate over decades_dic passing in the key and value to function get_sum_decade
decades = {key: get_sum_decade(df3, key, value) for key, value in decades_dic.items()}
print(f'Total de ocorrências por década: {decades}')
# create a dataframe from decades dict
df6 = pd.DataFrame.from_dict(decades, orient='index')
df6.reset_index(inplace=True)
df6.columns = ['Década', 'quant_oco_ano']
print(df6.head())

### Gráfico de barras

A partir do `df6` (contagens de ocorrências por década), vamos criar um gráfico de barras com plotly.

In [ ]:
# create a bar chart with the df6 dataframe
fig = px.bar(df6, x='Década', y='quant_oco_ano', color= 'Década', height=500, width=500,
                 hover_data=[df6['Década'], df6['quant_oco_ano']],
                 title=f'{output_file} - Década X Quantidade de ocorrências',
                 )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    legend=dict(orientation="v"),
    showlegend=False,
    xaxis=dict(title="Quantidade de ocorrências"),
    yaxis=dict(title="Décadas"),
    title_font_size=16,
    title_pad=dict(l=50, r=0, t=0, b=0),
)
# update xaxes
fig.update_xaxes(
        tickangle = 60,
        title_standoff = 25,
        nticks=20, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.update_yaxes(
        tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )

fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_decades_bar.html')
#save the figure as a html file
fig.write_html(fig_path)

## Analisar um único jornal

Vamos selecionar o acervo com maior número de ocorrências no dataframe e analisar

In [ ]:
# create a dataframe using df_acervos 
# select the first six rows of dataframe df_acervos
df7 = df_acervos.iloc[:6]
print(df7)
# get the values of 'Acrevo' column from df7 and put it in a list using .tolist()
lista_valores = df7['Acervo'].tolist()
print(f'Acervos: {lista_valores}')

In [ ]:
# use itens in `lista_valores` list to new dataframes
# strings must be defined manually
# filter dataframe by value in 'Acervo' column
acervo_1 = df3[df3['Acervo'].str.contains('Correio da Manhã ')]
print(acervo_1.head())
acervo_2 = df3[df3['Acervo'].str.contains('Jornal do Brasil ')]
print(acervo_2.head())
acervo_3 = df3[df3['Acervo'].str.contains('O Paiz ')]
print(acervo_3.head())
acervo_4 = df3[df3['Acervo'].str.contains('A Noite ')]
print(acervo_4.head())
acervo_5 = df3[df3['Acervo'].str.contains('Gazeta de Noticias ')]
print(acervo_5.head())
acervo_6 = df3[df3['Acervo'].str.contains('A Epoca ')]
print(acervo_6.head())
#

In [ ]:
# create a scatter plot with plotly express using x = 'Ano' and y = 'quant_oco_ano'
fig = px.scatter(acervo_1, x='Ano', y='quant_oco_ano', color='quant_oco_ano', height=600, width=800,
                 hover_data=[acervo_1['Ano'], acervo_1['Acervo'], acervo_1['quant_oco_ano']],
                 title=f'{output_file} - Correio da Manhã (RJ) - Ano X Acervo', # title must be defined manually
                 size='quant_oco_ano',
                 )
# config the layout
fig.update_layout(
    hovermode="closest",
    legend=dict(orientation="h", xanchor="center", yanchor="top"),
    showlegend=True,
    xaxis=dict(title="Anos"),
    yaxis=dict(title="Quantidade de ocorrências"),
    title_font_size=20,
    title_pad=dict(l=100, r=0, t=0, b=0)
)
# update xaxes
fig.update_xaxes(
        nticks=20, tickfont_size=10,
        tickangle = 60,
        title_standoff = 25,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.update_yaxes(
        nticks=20, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )

fig.show()
# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_acervo_1.html')
#save the figure as a html file
fig.write_html(fig_path)                


### Criar gráfico com várias figuras

In [ ]:
# creates a scatter plot with plotly using subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# define subplots and the number of rows and columns
fig = make_subplots(rows=2, cols=3)
# add scatter plots to the subplots
# the name of the scatter plot is the name of the list item and must be defined manually
fig.add_trace(go.Scatter(x=acervo_1['Ano'], y=acervo_1['quant_oco_ano'], name='Correio da Manhã (RJ)', mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=acervo_2['Ano'], y=acervo_2['quant_oco_ano'], name='Jornal do Brasil (RJ)', mode='lines+markers'), row=1, col=2)
fig.add_trace(go.Scatter(x=acervo_3['Ano'], y=acervo_3['quant_oco_ano'], name='O Paiz (RJ)', mode='lines+markers'), row=1, col=3)
fig.add_trace(go.Scatter(x=acervo_4['Ano'], y=acervo_4['quant_oco_ano'], name='A Noite (RJ)', mode='lines+markers'), row=2, col=1)
fig.add_trace(go.Scatter(x=acervo_5['Ano'], y=acervo_5['quant_oco_ano'], name='Gazeta de Noticias (RJ)', mode='lines+markers'), row=2, col=2)
fig.add_trace(go.Scatter(x=acervo_6['Ano'], y=acervo_6['quant_oco_ano'], name='A Epoca (RJ)', mode='lines+markers'), row=2, col=3)
fig.update_layout(
    #insert title
    title=f'{output_file} - Ano X Quantidade de ocorrências',
    height=600,
    width=800,
    showlegend=True,
    legend=dict(orientation="v", xanchor="center", yanchor="top"),
    title_font_size=20,
    title_pad=dict(l=100, r=0, t=0, b=0),
)
fig.update_xaxes(
        tickangle = 60,
        title_standoff = 25,
        ticks="outside", tickwidth=1,
        ticklen=5,
        nticks=10, tickfont_size=10,
        )
fig.update_yaxes(
        nticks=10, tickfont_size=10,
        ticks="outside", tickwidth=1,
        ticklen=5,
        )
fig.show()

# create a dir called 'figures'
# if it doesn't exist, create it
if not os.path.exists('./figures'):
    os.makedirs('./figures')
fig_path = os.path.join('./figures', f'hdb_{output_file}_mult_acervos.html')
#save the figure as a html file
fig.write_html(fig_path)
